## Define the Pairwise Dynamics of 2 Spherobots Using 
### (L, Theta1, Theta2) and (Hx, Hy, Theta_BW)

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib notebook
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [16]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
csfont = {'fontname':'Times New Roman'}

#Lists
#RLength
ThetaList=['0.0','22.5','45.0','67.5','90.0','112.5','135.0','157.5','180.0',
           '202.5','225.0','247.5','270.0','292.5','315.0','337.5']
HxList=['0.5','1.5','2.5','3.5','4.5','5.5','6.5','7.5','8.5','9.5','10.5','11.5','12.5']
HyList=['-13','-11','-9','-7','-5','-3','-1','1','3','5','7','9']

allData = []

#### Obtain Pos Data for all simulations
#### Limit time to first five oscillations

In [3]:
def StoreData(cwd_POSDATA, parHx, parHy, parTheta):
    #global axAll
    #Reset position data every Sim
    pdData = []
    #Load position data
    pdData = pd.read_csv(cwd_POSDATA+'/pd.txt',delimiter=' ')
    #Split up individual sphere data by given index
    UAdata = pdData[pdData['idx'] == 6].copy()
    LAdata = pdData[pdData['idx'] == 19].copy()
    UBdata = pdData[pdData['idx'] == 32].copy()
    LBdata = pdData[pdData['idx'] == 45].copy()
    #Sort data by time and reset indices
    UAdata = UAdata.sort_values(by=['time'])
    LAdata = LAdata.sort_values(by=['time'])
    UBdata = UBdata.sort_values(by=['time'])
    LBdata = LBdata.sort_values(by=['time'])
    UAdata = UAdata.reset_index(drop=True)
    LAdata = LAdata.reset_index(drop=True)
    UBdata = UBdata.reset_index(drop=True)
    LBdata = LBdata.reset_index(drop=True)
    #Rename columns to previous data frames
    UAdata = UAdata.rename(columns={"x":"aXU", "y":"aYU"})
    LAdata = LAdata.rename(columns={"x":"aXL", "y":"aYL"})
    UBdata = UBdata.rename(columns={"x":"bXU", "y":"bYU"})
    LBdata = LBdata.rename(columns={"x":"bXL", "y":"bYL"})
    #Combine separate dataframes to create previous dataframe used
    splitDict = {'aXU':UAdata['aXU'],'aYU':UAdata['aYU'],'aXL':LAdata['aXL'],'aYL':LAdata['aYL'],
                 'bXU':UBdata['bXU'],'bYU':UBdata['bYU'],'bXL':LBdata['bXL'],'bYL':LBdata['bYL'],'time':UAdata['time']}
    posData = pd.DataFrame(data=splitDict)
    #Calculate initial positioning given Hx, Hy and Theta
    #Add initial positioning
    initDict = {'aXU':[0.0],'aYU':[1.0e-3],'aXL':[0.0],'aYL':[-4.0e-3],
                'bXU':[parHx*RADIUSSMALL - 1.0e-3*np.sin(parTheta*np.pi/180.0)],'bYU':[parHy*RADIUSSMALL + 1.0e-3*np.cos(parTheta*np.pi/180.0)],
                'bXL':[parHx*RADIUSSMALL + 4.0e-3*np.sin(parTheta*np.pi/180.0)],'bYL':[parHy*RADIUSSMALL - 4.0e-3*np.cos(parTheta*np.pi/180.0)],
                'time':[0.0]}
    initData = pd.DataFrame(data=initDict)
    posData = posData.append(initData)
    posData = posData.sort_values(by=['time'])
    posData = posData.reset_index(drop=True)
    #Save only every 20 rows (Every period)
    posData = posData.iloc[::20]
    #Reindex so index corresponds to period number
    posData = posData.reset_index(drop=True)
    #Print pdData to make sure it has been done properly
    #if(parTheta == 22.5):
    #    print(posData.head(6))
    #sys.exit(0)
    #Create Swimmer A and Swimmer B dataframes
    dict_A = {'xU':posData['aXU'],'yU':posData['aYU'],'xL':posData['aXL'],'yL':posData['aYL'],'time':posData['time']}
    dataA  = pd.DataFrame(data=dict_A)
    dict_B = {'xU':posData['bXU'],'yU':posData['bYU'],'xL':posData['bXL'],'yL':posData['bYL'],'time':posData['time']}
    dataB  = pd.DataFrame(data=dict_B)
    #print(dataA)
    #print(dataB)
    
    #Calculate these three parameters (L, Theta1, and Theta2)
    #L is the distance from small-sphere to small-sphere
    #Theta1 is the angle between L and Swim1 axis
    #Theta2 is angle between L and Swim2 axis
    
    #Create a swimmer class for each
    #print('SwimA')
    swimA = Swimmer(dataA)
    #print('SwimB')
    swimB = Swimmer(dataB)
    angleA = swimA.get_theta()
    angleB = swimB.get_theta()
    #Calculate the three parameters (Hx, Hy, Theta_BW)
    #Hx, Hy, and theta_BW in swimmer A's reference frame
    Hx, Hy, theta_BW = swimB.CalcHxHyTheta(swimA)
    #Calculate the three parameters (L, Theta1, Theta2)
    LSS, theta1, theta2 = swimA.CalcLTheta1Theta2(swimB)
    '''
    print('Theta_A = ',angleA*180.0/np.pi)
    print('Theta_B = ',angleB*180.0/np.pi)
    print('LSSx = ',swimA.LSSx)
    print('LSSy = ',swimA.LSSy)
    print('LSS = ',LSS)
    print('ThetaLSS = ',swimA.thetaLSS*180.0/np.pi)
    print('Theta1 = ',theta1*180.0/np.pi)
    print('Theta2 = ',theta2*180.0/np.pi)
    sys.exit(0)
    '''
    #First, create a dataframe to contain this information
    dict_Param = {'Hx':Hx,'Hy':Hy,'ThetaBW':theta_BW,'LSS':LSS,'Theta1':theta1,'Theta2':theta2,'time':posData['time']}
    parData = pd.DataFrame(data=dict_Param)
    
    return parData


#### Find Angle Between Swimmers (Account for Quadrant)

In [4]:
def Rotate(xy, theta):
    # https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    #First Rotate based on Theta
    #Allocate Arrays
    rotationMatrix = np.zeros((2,2))
    #Calculate rotation matrix
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[0,1] = -1.0*np.sin(theta)
    rotationMatrix[1,0] = np.sin(theta)
    rotationMatrix[1,1] = np.cos(theta) 
    return rotationMatrix.dot(xy)

def Translate(xy, offset):
    return xy + offset

class Swimmer:
    def __init__(self,data):
        self.data  = data
        self.xU, self.yU = data['xU'], data['yU']
        self.xL, self.yL = data['xL'], data['yL']
        self.time = data['time']
        self.CM = np.zeros((2,len(self.time)))
        self.theta = np.zeros(len(self.time))
        self.normY = np.zeros((2,len(self.time)))
        self.normX = np.zeros((2,len(self.time)))
        self.Hx = np.zeros(len(self.time))
        self.Hy = np.zeros(len(self.time))
        self.rot_norm = np.zeros((2,len(self.time)))
        self.theta_BW = np.zeros(len(self.time))
        self.normLSS = np.zeros((2,len(self.time)))
        self.thetaLSS = np.zeros(len(self.time))
        self.rot_normLTT = np.zeros((2,len(self.time)))
        self.rot_normLSS = np.zeros((2,len(self.time)))
        self.theta1 = np.zeros(len(self.time))
        self.theta2 = np.zeros(len(self.time))
        self.CalcCM()
        self.CalcLabAngle()
    
    def get_theta(self):
        return self.theta
    
    def get_norms(self):
        return (self.normX, self.normY)
        
    def CalcCM(self):
        self.xCM = 0.8*self.xU + 0.2*self.xL
        self.yCM = 0.8*self.yU + 0.2*self.yL
        self.CM[0] = self.xCM
        self.CM[1] = self.yCM
        #print('CM = ',self.CM[:,0:10])
        
    def CalcLabAngle(self):
        #Find swimming axis (normal y-axis)
        self.labX   = self.xU - self.xL
        self.labY   = self.yU - self.yL
        self.length = np.hypot(self.labX,self.labY)
        self.normY[0] = self.labX/self.length
        self.normY[1] = self.labY/self.length
        #self.normY  = np.array([self.labX/self.length,self.labY,self.length])
        #2) Calculate Theta
        for idx in range(len(self.time)):
            if(self.normY[1,idx] >= 0.0):
                self.theta[idx] = np.arccos(self.normY[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.theta[idx] = -1.0*np.arccos(self.normY[0,idx])+2.0*np.pi
                #Theta_a[idx] = -1.0*np.arccos(norm_aX[idx])+2.0*np.pi
        #Calculate perpendicular axis (swimmer's x-axis)
        self.normX[0] = np.cos(self.theta - 0.5*np.pi)
        self.normX[1] = np.sin(self.theta - 0.5*np.pi)
        
    def CalcHxHyTheta(self,swimmer):
        #Here, we will calculate three parameters
        #1) Hx: In the swimmer A's reference frame
        #2) Hy: In the swimmer A's reference frame
        #3) Theta_BW: Angle between swimmer A and swimmerB (w/ respect to swimmer A y-axis)
        #But first, we must calculate in lab frame for some
        
        #1) Hx: In swimmer A's reference frame
        #2) Hy: In swimmer A's reference frame
        self.Projection(swimmer.normX,swimmer.normY,swimmer.CM)
        
        #3) Rotate A and B ccw by 2pi - Theta_a
        self.rotateAngle = 2.0*np.pi - swimmer.theta
        for idx in range(len(self.length)):
            self.rot_norm[:,idx] = Rotate(self.normY[:,idx],self.rotateAngle[idx])
            swimmer.rot_norm[:,idx] = Rotate(swimmer.normY[:,idx],self.rotateAngle[idx])
            if(self.rot_norm[1,idx] >= 0.0):
                self.theta_BW[idx] = np.arccos(swimmer.rot_norm[:,idx].dot(self.rot_norm[:,idx]))%(2.0*np.pi)
            else:
                self.theta_BW[idx] = (-1.0*np.arccos(swimmer.rot_norm[:,idx].dot(self.rot_norm[:,idx]))+2.0*np.pi)%(2.0*np.pi)
        
        return (self.Hx/RADIUSLARGE, self.Hy/RADIUSLARGE, self.theta_BW)
    
    def CalcLTheta1Theta2(self,swimmerB):
        #Here, we will calculate three parameters
        #1) LSS: Length between small spheres. We will find LSSx and LSSy norms for angle calculations
        #2) Theta1: Angle between LSS and swim1 axis (inside angle)
        #3) Theta2: Angle between LSS and swim2 axis (inside angle)
        
        #1) Find LSSx and LSSy
        #Calc LSS, normLSSx, and normLSSy
        
        self.LSSx = swimmerB.xL - self.xL
        self.LSSy = swimmerB.yL - self.yL
        self.LSS  = np.hypot(self.LSSx,self.LSSy)
        self.normLSS[0] = self.LSSx/self.LSS
        self.normLSS[1] = self.LSSy/self.LSS
        # Calculate thetaLSS
        for idx in range(len(self.time)):
            if(self.normLSS[1,idx] >= 0.0):
                self.thetaLSS[idx] = np.arccos(self.normLSS[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.thetaLSS[idx] = -1.0*np.arccos(self.normLSS[0,idx])+2.0*np.pi
        
        #2) Find Theta1: Angle between LSS and swim1 axis
        self.rotateAngleLSS = 2.0*np.pi - self.thetaLSS
        for idx in range(len(self.LSS)):
            self.rot_normLSS[:,idx] = Rotate(self.normLSS[:,idx],self.rotateAngleLSS[idx])
            self.rot_normLTT[:,idx] = Rotate(self.normY[:,idx],self.rotateAngleLSS[idx])
            swimmerB.rot_normLTT[:,idx] = Rotate(swimmerB.normY[:,idx],self.rotateAngleLSS[idx])

            if(self.rot_normLTT[1,idx] >= 0.0):
                self.theta1[idx] = np.arccos(self.rot_normLSS[:,idx].dot(self.rot_normLTT[:,idx]))%(2.0*np.pi)
                if(swimmerB.rot_normLTT[1,idx] >= 0.0):
                    self.theta2[idx] = np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))%(2.0*np.pi)
                else:
                    self.theta2[idx] = (-1.0*np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))+2.0*np.pi)%(2.0*np.pi)

            else:
                self.theta1[idx] = (-1.0*np.arccos(self.rot_normLSS[:,idx].dot(self.rot_normLTT[:,idx]))+2.0*np.pi)%(2.0*np.pi)
            
                if(swimmerB.rot_normLTT[1,idx] >= 0.0):
                    self.theta2[idx] = np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))%(2.0*np.pi)
                else:
                    self.theta2[idx] = (-1.0*np.arccos(-1.0*self.rot_normLSS[:,idx].dot(swimmerB.rot_normLTT[:,idx]))+2.0*np.pi)%(2.0*np.pi)
                self.theta1[idx] = 2.0*np.pi - self.theta1[idx]
                self.theta2[idx] = 2.0*np.pi - self.theta2[idx]
        
        return (self.LSS/RADIUSLARGE, self.theta1, self.theta2)

    def Projection(self,normX,normY,CM):
        #Here we find the distance component for swimmer A's axes
        #print('normY = ',normY)
        #print('CM = ',self.CM)
        for idx in range(len(self.time)):
            '''print('normY[:,idx] = ',normY[:,idx])
            print('CM[idx] = ',self.CM[:,idx])
            dot = np.dot(normY[:,idx],self.CM[:,idx])
            print('dot = ',dot)
            ycoord = dot*normY[:,idx]
            print('ycoord = ',ycoord)'''
            self.Hx[idx] = np.dot(normX[:,idx],(self.CM[:,idx]-CM[:,idx]))
            self.Hy[idx] = np.dot(normY[:,idx],(self.CM[:,idx]-CM[:,idx]))


#### Create dataframe of all pos data for all time for all sims

In [5]:
#The main goal of this script is to create an H-Theta Phase Space of all
#simulations for every period elapsed.
#Example: 20s of sim time = 200 periods. 200 H-Theta Plots
#We may find that we can combine the H-Theta data after steady state has been reached
#1) For each simulation, store position data for every period (nothing in between)
#2) Calculate separation distance between large spheres (H)
#3) Calculate relative heading (Theta)
#4) Calculate deltaH and deltaTheta (change after one Period)
#5) Plot H vs Theta (Polar) for each period

nTheta = len(ThetaList)
nHx = len(HxList)
nHy = len(HyList)

count = 0
dict_allData = {'Hx':[],'Hy':[],'ThetaBW':[],'parHx':[],'parHy':[],'parThetaBW':[],'time':[]}
df_allData = pd.DataFrame(data=dict_allData)
for Theta in ThetaList:
    for Hx in HxList:
        dirNames = [ name for name in os.listdir(cwd_PYTHON+'/../PosData/Theta'+Theta+'/Hx'+Hx+'/') 
                    if os.path.isdir(os.path.join(cwd_PYTHON+'/../PosData/Theta'+Theta+'/Hx'+Hx+'/', name)) ]
        print('dirHys = ',dirNames)
        for dirHy in dirNames:
            cwd_POSDATA = cwd_PYTHON+'/../PosData/Theta'+Theta+'/Hx'+Hx+'/'+dirHy
            HyString, HyValue = tuple(re.split('Hy',dirHy)[:2])
            strPrint = 'Theta:'+Theta+'\tHx:'+Hx+'\t'+dirHy
            print(strPrint)
            #Get All sim data and store
            simData = StoreData(cwd_POSDATA,float(Hx),float(HyValue),float(Theta))
            #Add Simulation Parameter identifiers
            simData['parThetaBW'], simData['parHx'], simData['parHy'] = 180.0*simData.loc[0,'ThetaBW']/np.pi, simData.loc[0,'Hx'], simData.loc[0,'Hy']
            simData['parLSS'], simData['parTheta1'], simData['parTheta2'] = simData.loc[0,'LSS'], round(simData.loc[0,'Theta1']*180.0/np.pi,1), round(simData.loc[0,'Theta2']*180.0/np.pi,1)
            strPrint = 'LSS:%.1f \tTheta1: %.1f\tTheta2: %.1f'%(simData.loc[0,'parLSS'],simData.loc[0,'parTheta1'],simData.loc[0,'parTheta2'])
            print(strPrint)
            dict_sim = {'parHx':simData['parHx'],'parHy':simData['parHy'],'parThetaBW':simData['parThetaBW'],
                        'parLSS':simData['parLSS'],'parTheta1':simData['parTheta1'],'parTheta2':simData['parTheta2'],
                        'Hx':simData['Hx'],'Hy':simData['Hy'],'ThetaBW':simData['ThetaBW'],
                        'LSS':simData['LSS'],'Theta1':simData['Theta1'],'Theta2':simData['Theta2'],
                        'time':simData['time']}
            df_sim = pd.DataFrame(data=dict_sim)
            df_allData = pd.concat([df_allData,df_sim],ignore_index=True)
            count += 1
    print('Theta ='+Theta)
allData = df_allData.copy()
allData = allData.sort_values(by=['parThetaBW','parHx','parHy','time'])
allData['H'] = np.hypot(allData['Hx'],allData['Hy'])
allData.to_csv(cwd_PYTHON+'/LTT_allData_Re10.csv',index=False,sep=' ')
print(len(allData['time']))
print('count = ',count)
print('Storing Data is complete! Onto Analysis')
                

dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy9']
Theta:0.0	Hx:0.5	Hy-11
LSS:5.5 	Theta1: 177.4	Theta2: 2.6
Theta:0.0	Hx:0.5	Hy-13
LSS:6.5 	Theta1: 177.8	Theta2: 2.2
Theta:0.0	Hx:0.5	Hy-9
LSS:4.5 	Theta1: 176.8	Theta2: 3.2
Theta:0.0	Hx:0.5	Hy9


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



LSS:4.5 	Theta1: 3.2	Theta2: 176.8
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy9']
Theta:0.0	Hx:1.5	Hy-11
LSS:5.6 	Theta1: 172.2	Theta2: 7.8
Theta:0.0	Hx:1.5	Hy-13
LSS:6.5 	Theta1: 173.4	Theta2: 6.6
Theta:0.0	Hx:1.5	Hy-9
LSS:4.6 	Theta1: 170.5	Theta2: 9.5
Theta:0.0	Hx:1.5	Hy9
LSS:4.6 	Theta1: 9.5	Theta2: 170.5
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy9']
Theta:0.0	Hx:2.5	Hy-11
LSS:5.6 	Theta1: 167.2	Theta2: 12.8
Theta:0.0	Hx:2.5	Hy-13
LSS:6.6 	Theta1: 169.1	Theta2: 10.9
Theta:0.0	Hx:2.5	Hy-9
LSS:4.7 	Theta1: 164.5	Theta2: 15.5
Theta:0.0	Hx:2.5	Hy9
LSS:4.7 	Theta1: 15.5	Theta2: 164.5
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy9']
Theta:0.0	Hx:3.5	Hy-11
LSS:5.8 	Theta1: 162.3	Theta2: 17.7
Theta:0.0	Hx:3.5	Hy-13
LSS:6.7 	Theta1: 164.9	Theta2: 15.1
Theta:0.0	Hx:3.5	Hy-9
LSS:4.8 	Theta1: 158.7	Theta2: 21.3
Theta:0.0	Hx:3.5	Hy9
LSS:4.8 	Theta1: 21.3	Theta2: 158.7
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:0.0	Hx:4.5	Hy-1
LSS:2.3 	The

LSS:6.5 	Theta1: 166.6	Theta2: 350.9
Theta:22.5	Hx:1.5	Hy-9
LSS:4.6 	Theta1: 160.8	Theta2: 356.7
Theta:22.5	Hx:1.5	Hy9
LSS:4.9 	Theta1: 18.0	Theta2: 139.5
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy9']
Theta:22.5	Hx:2.5	Hy-11
LSS:5.7 	Theta1: 159.4	Theta2: 358.1
Theta:22.5	Hx:2.5	Hy-13
LSS:6.7 	Theta1: 162.4	Theta2: 355.1
Theta:22.5	Hx:2.5	Hy-9
LSS:4.8 	Theta1: 155.1	Theta2: 2.4
Theta:22.5	Hx:2.5	Hy9
LSS:5.1 	Theta1: 23.4	Theta2: 134.1
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy5', 'Hy7', 'Hy9']
Theta:22.5	Hx:3.5	Hy-11
LSS:5.9 	Theta1: 154.8	Theta2: 2.7
Theta:22.5	Hx:3.5	Hy-13
LSS:6.8 	Theta1: 158.4	Theta2: 359.1
Theta:22.5	Hx:3.5	Hy-9
LSS:5.0 	Theta1: 149.9	Theta2: 7.6
Theta:22.5	Hx:3.5	Hy5
LSS:3.7 	Theta1: 43.5	Theta2: 114.0
Theta:22.5	Hx:3.5	Hy7
LSS:4.4 	Theta1: 34.6	Theta2: 122.9
Theta:22.5	Hx:3.5	Hy9
LSS:5.3 	Theta1: 28.4	Theta2: 129.1
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:22.5	Hx:4.5	Hy-11
LSS:6.1 	Theta1: 150.6	Theta2: 6.9
Theta:22.5	Hx:4.5	Hy-13
LSS:7

LSS:4.5 	Theta1: 151.1	Theta2: 343.9
Theta:45.0	Hx:1.5	Hy7
LSS:4.6 	Theta1: 27.9	Theta2: 107.1
Theta:45.0	Hx:1.5	Hy9
LSS:5.5 	Theta1: 23.1	Theta2: 111.9
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy7', 'Hy9']
Theta:45.0	Hx:2.5	Hy-11
LSS:5.6 	Theta1: 151.5	Theta2: 343.5
Theta:45.0	Hx:2.5	Hy-13
LSS:6.5 	Theta1: 155.7	Theta2: 339.3
Theta:45.0	Hx:2.5	Hy-9
LSS:4.7 	Theta1: 145.8	Theta2: 349.2
Theta:45.0	Hx:2.5	Hy7
LSS:4.9 	Theta1: 33.1	Theta2: 101.9
Theta:45.0	Hx:2.5	Hy9
LSS:5.7 	Theta1: 27.6	Theta2: 107.4
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy5', 'Hy7', 'Hy9']
Theta:45.0	Hx:3.5	Hy-11
LSS:5.8 	Theta1: 147.2	Theta2: 347.8
Theta:45.0	Hx:3.5	Hy-13
LSS:6.7 	Theta1: 151.9	Theta2: 343.1
Theta:45.0	Hx:3.5	Hy-9
LSS:5.0 	Theta1: 141.0	Theta2: 354.0
Theta:45.0	Hx:3.5	Hy5
LSS:4.4 	Theta1: 45.7	Theta2: 89.3
Theta:45.0	Hx:3.5	Hy7
LSS:5.2 	Theta1: 37.8	Theta2: 97.2
Theta:45.0	Hx:3.5	Hy9
LSS:6.0 	Theta1: 31.9	Theta2: 103.1
dirHys =  ['Hy-11', 'Hy-13', 'Hy-7', 'Hy-9', 'Hy5', 'Hy7']
Theta:45.0	Hx:4.5	Hy-11
LSS

...

LSS:5.3 	Theta1: 9.7	Theta2: 282.8
Theta:247.5	Hx:5.5	Hy7
LSS:6.3 	Theta1: 8.2	Theta2: 284.3
Theta:247.5	Hx:5.5	Hy9
LSS:7.3 	Theta1: 7.1	Theta2: 285.4
dirHys =  ['Hy-11', 'Hy-13', 'Hy-9', 'Hy5', 'Hy7', 'Hy9']
Theta:247.5	Hx:6.5	Hy-11
LSS:3.1 	Theta1: 152.9	Theta2: 139.6
Theta:247.5	Hx:6.5	Hy-13
LSS:4.0 	Theta1: 159.4	Theta2: 133.1
Theta:247.5	Hx:6.5	Hy-9
LSS:2.2 	Theta1: 141.0	Theta2: 151.5
Theta:247.5	Hx:6.5	Hy5
LSS:5.4 	Theta1: 14.9	Theta2: 277.6
Theta:247.5	Hx:6.5	Hy7
LSS:6.4 	Theta1: 12.6	Theta2: 279.9
Theta:247.5	Hx:6.5	Hy9
LSS:7.4 	Theta1: 10.9	Theta2: 281.6
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:247.5	Hx:7.5	Hy-1
LSS:3.0 	Theta1: 40.0	Theta2: 252.5
Theta:247.5	Hx:7.5	Hy-11
LSS:3.3 	Theta1: 145.2	Theta2: 147.3
Theta:247.5	Hx:7.5	Hy-13
LSS:4.2 	Theta1: 153.0	Theta2: 139.5
Theta:247.5	Hx:7.5	Hy-3
LSS:2.3 	Theta1: 56.4	Theta2: 236.1
Theta:247.5	Hx:7.5	Hy-5
LSS:1.9 	Theta1: 82.1	Theta2: 210.4
Theta:247.5	Hx:7.5	Hy

LSS:4.8 	Theta1: 158.7	Theta2: 111.3
Theta:270.0	Hx:7.5	Hy-3
LSS:1.8 	Theta1: 74.1	Theta2: 195.9
Theta:270.0	Hx:7.5	Hy-5
LSS:1.8 	Theta1: 105.9	Theta2: 164.1
Theta:270.0	Hx:7.5	Hy-7
LSS:2.3 	Theta1: 130.6	Theta2: 139.4
Theta:270.0	Hx:7.5	Hy-9
LSS:3.1 	Theta1: 145.0	Theta2: 125.0
Theta:270.0	Hx:7.5	Hy1
LSS:3.1 	Theta1: 35.0	Theta2: 235.0
Theta:270.0	Hx:7.5	Hy3
LSS:3.9 	Theta1: 26.6	Theta2: 243.4
Theta:270.0	Hx:7.5	Hy5
LSS:4.8 	Theta1: 21.3	Theta2: 248.7
Theta:270.0	Hx:7.5	Hy7
LSS:5.8 	Theta1: 17.7	Theta2: 252.3
Theta:270.0	Hx:7.5	Hy9
LSS:6.7 	Theta1: 15.1	Theta2: 254.9
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:270.0	Hx:8.5	Hy-1
LSS:2.7 	Theta1: 56.3	Theta2: 213.7
Theta:270.0	Hx:8.5	Hy-11
LSS:4.2 	Theta1: 147.3	Theta2: 122.7
Theta:270.0	Hx:8.5	Hy-13
LSS:5.0 	Theta1: 153.4	Theta2: 116.6
Theta:270.0	Hx:8.5	Hy-3
LSS:2.3 	Theta1: 77.5	Theta2: 192.5
Theta:270.0	Hx:8.5	Hy-5
LSS:2.3 	Theta1: 102.5	Theta2: 167.5
Theta:270.0	Hx:8

LSS:5.1 	Theta1: 21.9	Theta2: 225.6
Theta:292.5	Hx:7.5	Hy9
LSS:6.0 	Theta1: 18.4	Theta2: 229.1
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:292.5	Hx:8.5	Hy-1
LSS:2.5 	Theta1: 73.0	Theta2: 174.5
Theta:292.5	Hx:8.5	Hy-11
LSS:4.9 	Theta1: 150.6	Theta2: 96.9
Theta:292.5	Hx:8.5	Hy-13
LSS:5.8 	Theta1: 155.5	Theta2: 92.0
Theta:292.5	Hx:8.5	Hy-3
LSS:2.4 	Theta1: 96.3	Theta2: 151.2
Theta:292.5	Hx:8.5	Hy-5
LSS:2.7 	Theta1: 117.8	Theta2: 129.7
Theta:292.5	Hx:8.5	Hy-7
LSS:3.3 	Theta1: 133.3	Theta2: 114.2
Theta:292.5	Hx:8.5	Hy-9
LSS:4.1 	Theta1: 143.7	Theta2: 103.8
Theta:292.5	Hx:8.5	Hy1
LSS:3.0 	Theta1: 54.2	Theta2: 193.3
Theta:292.5	Hx:8.5	Hy3
LSS:3.6 	Theta1: 41.3	Theta2: 206.2
Theta:292.5	Hx:8.5	Hy5
LSS:4.4 	Theta1: 32.8	Theta2: 214.7
Theta:292.5	Hx:8.5	Hy7
LSS:5.3 	Theta1: 26.9	Theta2: 220.6
Theta:292.5	Hx:8.5	Hy9
LSS:6.2 	Theta1: 22.7	Theta2: 224.8
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3'

LSS:5.0 	Theta1: 34.8	Theta2: 190.2
Theta:315.0	Hx:8.5	Hy9
LSS:5.8 	Theta1: 29.1	Theta2: 195.9
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:315.0	Hx:9.5	Hy-1
LSS:3.3 	Theta1: 88.5	Theta2: 136.5
Theta:315.0	Hx:9.5	Hy-11
LSS:5.9 	Theta1: 145.8	Theta2: 79.2
Theta:315.0	Hx:9.5	Hy-13
LSS:6.8 	Theta1: 150.6	Theta2: 74.4
Theta:315.0	Hx:9.5	Hy-3
LSS:3.5 	Theta1: 105.3	Theta2: 119.7
Theta:315.0	Hx:9.5	Hy-5
LSS:3.8 	Theta1: 119.8	Theta2: 105.2
Theta:315.0	Hx:9.5	Hy-7
LSS:4.4 	Theta1: 131.1	Theta2: 93.9
Theta:315.0	Hx:9.5	Hy-9
LSS:5.1 	Theta1: 139.6	Theta2: 85.4
Theta:315.0	Hx:9.5	Hy1
LSS:3.5 	Theta1: 72.0	Theta2: 153.0
Theta:315.0	Hx:9.5	Hy3
LSS:3.9 	Theta1: 58.0	Theta2: 167.0
Theta:315.0	Hx:9.5	Hy5
LSS:4.5 	Theta1: 47.2	Theta2: 177.8
Theta:315.0	Hx:9.5	Hy7
LSS:5.3 	Theta1: 39.2	Theta2: 185.8
Theta:315.0	Hx:9.5	Hy9
LSS:6.1 	Theta1: 33.3	Theta2: 191.7
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3',

LSS:7.5 	Theta1: 147.9	Theta2: 54.6
Theta:337.5	Hx:9.5	Hy-3
LSS:4.2 	Theta1: 108.7	Theta2: 93.8
Theta:337.5	Hx:9.5	Hy-5
LSS:4.6 	Theta1: 120.5	Theta2: 82.0
Theta:337.5	Hx:9.5	Hy-7
LSS:5.2 	Theta1: 130.0	Theta2: 72.5
Theta:337.5	Hx:9.5	Hy-9
LSS:5.9 	Theta1: 137.5	Theta2: 65.0
Theta:337.5	Hx:9.5	Hy1
LSS:4.0 	Theta1: 80.7	Theta2: 121.8
Theta:337.5	Hx:9.5	Hy3
LSS:4.3 	Theta1: 67.5	Theta2: 135.0
Theta:337.5	Hx:9.5	Hy5
LSS:4.8 	Theta1: 56.4	Theta2: 146.1
Theta:337.5	Hx:9.5	Hy7
LSS:5.4 	Theta1: 47.5	Theta2: 155.0
Theta:337.5	Hx:9.5	Hy9
LSS:6.1 	Theta1: 40.6	Theta2: 161.9
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:337.5	Hx:10.5	Hy-1
LSS:4.5 	Theta1: 94.4	Theta2: 108.1
Theta:337.5	Hx:10.5	Hy-11
LSS:7.0 	Theta1: 140.0	Theta2: 62.5
Theta:337.5	Hx:10.5	Hy-13
LSS:7.8 	Theta1: 144.8	Theta2: 57.7
Theta:337.5	Hx:10.5	Hy-3
LSS:4.7 	Theta1: 106.7	Theta2: 95.8
Theta:337.5	Hx:10.5	Hy-5
LSS:5.1 	Theta1: 117.6	Theta2: 84.9
Theta:337.5	Hx:10.

#### Plot The Phase Space in 2D plane slices
##### (Hx, Hy) (H, Theta)

In [13]:
import matplotlib
def WhichEndState(data,scheme):
    #In this function, we will determine if the pair are in an end state configuration
    #Diverge = 0: Red
    #V-Shape = 1: Blue
    #Orbit = 2: pink
    #NOTA = 3: black
    
    #Diverge: H_bw > 10
    #V-Shape: Theta = 29 or 331; H_bw ~ 3.0R
    #Headbutt: Theta = 180; H_bw = 3.2-3.4R
    #Orbit: Theta = 180; H_bw = 3.0-3.25R
    
    #data = data.sort_values(by=['parHx','parHy','time'])
    data = data.reset_index(drop=True)
    data['State'] = 4
    data['color'] = 'black'
    data['Theta_deg'] = 180.0*data['ThetaBW']/np.pi
    data['color_r'] = 0
    data['color_g'] = 0
    data['color_b'] = 0
    color_value = matplotlib.colors.to_rgb('black')
    for idx in range(len(data['time'])):
        #Which End State?
        #Diverge
        if(data.loc[idx,'H'] > 10.0):
            data.loc[idx,'State'] = 0
            data.loc[idx,'color'] = 'gray'
            color_value = matplotlib.colors.to_rgb('gray')
            data.loc[idx,'color_r'] = color_value[0]
            data.loc[idx,'color_g'] = color_value[1]
            data.loc[idx,'color_b'] = color_value[2]
        #V-Shape
        elif((data.loc[idx,'H'] < 3.1 and data.loc[idx,'H'] >= 2.8) and 
             ((data.loc[idx,'Theta_deg'] < 32.0 and data.loc[idx,'Theta_deg'] >= 26.0) or
              (data.loc[idx,'Theta_deg'] < 334.0 and data.loc[idx,'Theta_deg'] >= 328.0))):
            data.loc[idx,'State'] = 1
            data.loc[idx,'color'] = 'blue'
            color_value = matplotlib.colors.to_rgb('blue')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 141
                data.loc[idx,'color_g'] = 160
                data.loc[idx,'color_b'] = 203
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 117
                data.loc[idx,'color_g'] = 112
                data.loc[idx,'color_b'] = 179
        #Orbit
        elif((data.loc[idx,'H'] < 3.25 and data.loc[idx,'H'] >= 3.0) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) ):
            data.loc[idx,'State'] = 2
            data.loc[idx,'color'] = 'pink'
            color_value = matplotlib.colors.to_rgb('pink')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 231
                data.loc[idx,'color_g'] = 138
                data.loc[idx,'color_b'] = 195
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 231
                data.loc[idx,'color_g'] = 41
                data.loc[idx,'color_b'] = 138
        #Headbutt
        elif((data.loc[idx,'H'] < 3.5 and data.loc[idx,'H'] >= 3.25) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) and
             (data.loc[idx,'Hx'] <= 0.1 and data.loc[idx,'Hx'] > -0.1)):
            data.loc[idx,'State'] = 3
            data.loc[idx,'color'] = 'orange'
            color_value = matplotlib.colors.to_rgb('orange')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 252
                data.loc[idx,'color_g'] = 131
                data.loc[idx,'color_b'] = 98
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 217
                data.loc[idx,'color_g'] = 95
                data.loc[idx,'color_b'] = 2
        else:
            color_value= matplotlib.colors.to_rgb('black')
        #data.loc[idx,'color_r'] = color_value[0]
        #data.loc[idx,'color_g'] = color_value[1]
        #data.loc[idx,'color_b'] = color_value[2]
    
    return data

### PLOT IN 3D PHASE DIAGRAM: Hx, Hy, ThetaBW

In [8]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/LTT_allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parHx'] >= 0.0].copy()
endData = endData.sort_values(by=['parTheta','parHx','parHy'])
endData = endData.reset_index(drop=True)
endData['xval'] = 0.5*endData['parHx']*np.cos(endData['parTheta']*np.pi/180.0)
endData['yval'] = 0.5*endData['parHx']*np.sin(endData['parTheta']*np.pi/180.0)
endData['zval'] = 0.5*endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
scheme='dark'
endData = WhichEndState(endData,scheme)

endData['ColorString'] = 0.0
for idx in range(len(endData['color_r'])):
    endData.loc[idx,'ColorString'] = 'rgb({0},{1},{2})'.format(endData.loc[idx,'color_r'],endData.loc[idx,'color_g'],endData.loc[idx,'color_b'])

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#NOTA = 3: black
    
#Create traces for each end state
traces = [0 for a in range(7)]
opaque = 0.4

#Divergence
divergeData = endData[endData['State'] == 0].copy()
divergeData = divergeData.reset_index(drop=True)

div1Data = divergeData[divergeData['zval'] >= 0.0].copy()
div1Data = div1Data.reset_index(drop=True)
div2Data = divergeData[divergeData['zval'] < 0.0].copy()
div2Data = div2Data.reset_index(drop=True)
div3Data = div2Data[div2Data['xval'] >= 0.0].copy()
div3Data = div3Data.reset_index(drop=True)
div4Data = div2Data[div2Data['xval'] < 0.0].copy()
div4Data = div4Data.reset_index(drop=True)

print(div1Data.loc[0,'ColorString'])

traces[0] = go.Mesh3d(x=div1Data['xval'], 
                     y=div1Data['yval'], 
                     z=div1Data['zval'], 
                     #color=div1Data.loc[0,'color'],
                     color=div1Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=2.25
                     )

traces[1] = go.Mesh3d(x=div3Data['xval'], 
                     y=div3Data['yval'], 
                     z=div3Data['zval'], 
                     #color=div3Data.loc[0,'color'], 
                     color=div3Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=1.61
                     )

traces[2] = go.Mesh3d(x=div4Data['xval'], 
                     y=div4Data['yval'], 
                     z=div4Data['zval'], 
                     #color=div4Data.loc[0,'color'], 
                     color=div4Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=3
                     )

#V-Shape
VshapeData = endData[endData['State'] == 1].copy()
VshapeData = VshapeData.reset_index(drop=True)
traces[3] = go.Mesh3d(x=VshapeData['xval'], 
                     y=VshapeData['yval'], 
                     z=VshapeData['zval'], 
                     #color=VshapeData.loc[0,'color'], 
                     color=VshapeData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

#Orbit
orbitData = endData[endData['State'] == 2].copy()
orbitData = orbitData.reset_index(drop=True)
traces[4] = go.Mesh3d(x=orbitData['xval'], 
                     y=orbitData['yval'], 
                     z=orbitData['zval'], 
                     #color=orbitData.loc[0,'color'], 
                     color=orbitData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

#Headbutt
headbuttData = endData[endData['State'] == 3].copy()
headbuttData = headbuttData.reset_index(drop=True)
traces[5] = go.Mesh3d(x=headbuttData['xval'], 
                     y=headbuttData['yval'], 
                     z=headbuttData['zval'], 
                     #color=headbuttData.loc[0,'color'], 
                     color=headbuttData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=5
                     )

traces[6] = go.Scatter3d(
    x=endData['xval'],
    y=endData['yval'],
    z=endData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=12,
        #color=endData['color'],                # set color to an array/list of desired values
        color=endData['ColorString'],
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

layout = go.Layout(
    #title='Re10: 3D Phase Diagram',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        )
    ),
    showlegend=False,
)

fig = go.Figure(data = traces, layout=layout)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene=dict(
                    #xaxis_title=r'Hx*cos(theta) (R)',
                    #yaxis_title=r'Hx*sin(theta) (R)',
                    #zaxis_title=r'Hy (R)'),
                    #xaxis_title='H<sub>x<\sub>cos(&theta) (R)',
                    #yaxis_title='H<sub>x<\sub>sin(&theta) (R)',
                    #zaxis_title='H<sub>y<\sub> (R)',
                    xaxis_title=dict(text="H<sub>x</sub> cos("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    yaxis_title=dict(text="H<sub>x</sub> sin("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    zaxis_title=dict(text="H<sub>y</sub> (R)",font=dict(size=18,family='Times New Roman')),
                    ),
                    width=2000,height=2000,
                    margin=dict(r=100, b=100, l=100, t=100),
                    font=dict(
                            family="Times New Roman",
                            size=12,
                            color="black"
                            )
            )
fig.update_xaxes(automargin=True)
fig.update_yaxes(automargin=True)
#fig.update_zaxes(automargin=True)

camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()

if not os.path.exists(cwd_PYTHON+'/../Figures/3D_Diagram'):
    os.mkdir(cwd_PYTHON+'/../Figures/3D_Diagram')
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_topview.png')
print('Done Making Top View')
camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=-2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_botview.png')
print('Done Making Bot View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=2.0, z=0.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_yposview.png')
print('Done Making Ypos View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=-2.0, z=0.0),
    #center=dict(x=0, y=0, z=1.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_ynegview.png')
print('Done Making Yneg View')


rgb(0.5019607843137255,0.5019607843137255,0.5019607843137255)
Done Making Top View
Done Making Bot View
Done Making Ypos View
Done Making Yneg View


### PLOT IN 3D PHASE DIAGRAM: L, Theta1, Theta2
#### WILL BE A SPHERE

In [14]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/LTT_allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
#endData = endData[endData['parTheta2'] <= 180.0].copy()
endData = endData.sort_values(by=['parLSS','parTheta2','parTheta1'])
endData = endData.reset_index(drop=True)
print('b4: 1: ',np.amax(endData['parTheta1']))
print('b4: 2: ',np.amax(endData['parTheta2']))
print('b4: BW: ',np.amin(endData['parThetaBW']))
print('b4: BW: ',np.amax(endData['parThetaBW']))
#Theta2 = phi, Theta1 = theta
'''
for idx in range(len(endData['parTheta1'])):
    if(endData.loc[idx,'parTheta1'] > 180.0):
        endData.loc[idx,'parTheta1'] = 360.0 - endData.loc[idx,'parTheta1']
        endData.loc[idx,'parTheta2'] = 360.0 - endData.loc[idx,'parTheta2']
endData = endData.sort_values(by=['parLSS','parTheta2','parTheta1'])
endData = endData.reset_index(drop=True)
print('a4: 1: ',np.amax(endData['parTheta1']))
print('a4: 2: ',np.amax(endData['parTheta2']))
sys.exit(0)
'''
#Convert to spherical coordinates
endData['xval'] = endData['parLSS']*np.cos(endData['parTheta2']*np.pi/180.0)*np.sin(endData['parTheta1']*np.pi/180.0)
endData['yval'] = endData['parLSS']*np.sin(endData['parTheta2']*np.pi/180.0)*np.sin(endData['parTheta1']*np.pi/180.0)
endData['zval'] = endData['parLSS']*np.cos(endData['parTheta1']*np.pi/180.0)
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
scheme='dark'
endData = WhichEndState(endData,scheme)

endData['ColorString'] = 0.0
for idx in range(len(endData['color_r'])):
    endData.loc[idx,'ColorString'] = 'rgb({0},{1},{2})'.format(endData.loc[idx,'color_r'],endData.loc[idx,'color_g'],endData.loc[idx,'color_b'])

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#NOTA = 3: black
    
#Create traces for each end state
traces = [0 for a in range(7)]
opaque = 0.4

#Divergence
divergeData = endData[endData['State'] == 0].copy()
divergeData = divergeData.reset_index(drop=True)

div1Data = divergeData[divergeData['zval'] >= 0.0].copy()
div1Data = div1Data.reset_index(drop=True)
div2Data = divergeData[divergeData['zval'] < 0.0].copy()
div2Data = div2Data.reset_index(drop=True)
div3Data = div2Data[div2Data['xval'] >= 0.0].copy()
div3Data = div3Data.reset_index(drop=True)
div4Data = div2Data[div2Data['xval'] < 0.0].copy()
div4Data = div4Data.reset_index(drop=True)

print(div1Data.loc[0,'ColorString'])

traces[0] = go.Mesh3d(x=div1Data['xval'], 
                     y=div1Data['yval'], 
                     z=div1Data['zval'], 
                     #color=div1Data.loc[0,'color'],
                     color=div1Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=2.25
                     )

traces[1] = go.Mesh3d(x=div3Data['xval'], 
                     y=div3Data['yval'], 
                     z=div3Data['zval'], 
                     #color=div3Data.loc[0,'color'], 
                     color=div3Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=1.61
                     )

traces[2] = go.Mesh3d(x=div4Data['xval'], 
                     y=div4Data['yval'], 
                     z=div4Data['zval'], 
                     #color=div4Data.loc[0,'color'], 
                     color=div4Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=3
                     )

#V-Shape
VshapeData = endData[endData['State'] == 1].copy()
VshapeData = VshapeData.reset_index(drop=True)
traces[3] = go.Mesh3d(x=VshapeData['xval'], 
                     y=VshapeData['yval'], 
                     z=VshapeData['zval'], 
                     #color=VshapeData.loc[0,'color'], 
                     color=VshapeData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

#Orbit
orbitData = endData[endData['State'] == 2].copy()
orbitData = orbitData.reset_index(drop=True)
traces[4] = go.Mesh3d(x=orbitData['xval'], 
                     y=orbitData['yval'], 
                     z=orbitData['zval'], 
                     #color=orbitData.loc[0,'color'], 
                     color=orbitData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

#Headbutt
headbuttData = endData[endData['State'] == 3].copy()
headbuttData = headbuttData.reset_index(drop=True)
traces[5] = go.Mesh3d(x=headbuttData['xval'], 
                     y=headbuttData['yval'], 
                     z=headbuttData['zval'], 
                     #color=headbuttData.loc[0,'color'], 
                     color=headbuttData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=5
                     )

traces[6] = go.Scatter3d(
    x=endData['xval'],
    y=endData['yval'],
    z=endData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=12,
        #color=endData['color'],                # set color to an array/list of desired values
        color=endData['ColorString'],
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

layout = go.Layout(
    #title='Re10: 3D Phase Diagram',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        )
    ),
    showlegend=False,
)

fig = go.Figure(data = traces, layout=layout)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene=dict(
                    #xaxis_title=r'Hx*cos(theta) (R)',
                    #yaxis_title=r'Hx*sin(theta) (R)',
                    #zaxis_title=r'Hy (R)'),
                    #xaxis_title='H<sub>x<\sub>cos(&theta) (R)',
                    #yaxis_title='H<sub>x<\sub>sin(&theta) (R)',
                    #zaxis_title='H<sub>y<\sub> (R)',
                    xaxis_title=dict(text="H<sub>x</sub> cos("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    yaxis_title=dict(text="H<sub>x</sub> sin("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    zaxis_title=dict(text="H<sub>y</sub> (R)",font=dict(size=18,family='Times New Roman')),
                    ),
                    width=2000,height=2000,
                    margin=dict(r=100, b=100, l=100, t=100),
                    font=dict(
                            family="Times New Roman",
                            size=12,
                            color="black"
                            )
            )
fig.update_xaxes(automargin=True)
fig.update_yaxes(automargin=True)
#fig.update_zaxes(automargin=True)

camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()

if not os.path.exists(cwd_PYTHON+'/../Figures/3D_Diagram'):
    os.mkdir(cwd_PYTHON+'/../Figures/3D_Diagram')
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re10_topview.png')
print('Done Making Top View')
camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=-2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re10_botview.png')
print('Done Making Bot View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=2.0, z=0.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re10_yposview.png')
print('Done Making Ypos View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=-2.0, z=0.0),
    #center=dict(x=0, y=0, z=1.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re10_ynegview.png')
print('Done Making Yneg View')


b4: 1:  179.9
b4: 2:  359.7
b4: BW:  0.0
b4: BW:  337.50000000000006
rgb(0.5019607843137255,0.5019607843137255,0.5019607843137255)
Done Making Top View
Done Making Bot View
Done Making Ypos View
Done Making Yneg View


In [96]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/LTT_allData_Re2.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parTheta2'] <= 180.0].copy()
endData = endData.sort_values(by=['parLSS','parTheta2','parTheta1'])
endData = endData.reset_index(drop=True)
print('b4: 1: ',np.amax(endData['parTheta1']))
print('b4: 2: ',np.amax(endData['parTheta2']))
print('b4: BW: ',np.amin(endData['parThetaBW']))
print('b4: BW: ',np.amax(endData['parThetaBW']))
#Theta2 = phi, Theta1 = theta
'''
for idx in range(len(endData['parTheta1'])):
    if(endData.loc[idx,'parTheta1'] > 180.0):
        endData.loc[idx,'parTheta1'] = 360.0 - endData.loc[idx,'parTheta1']
        endData.loc[idx,'parTheta2'] = 360.0 - endData.loc[idx,'parTheta2']
endData = endData.sort_values(by=['parLSS','parTheta2','parTheta1'])
endData = endData.reset_index(drop=True)
print('a4: 1: ',np.amax(endData['parTheta1']))
print('a4: 2: ',np.amax(endData['parTheta2']))
sys.exit(0)
'''
#Convert to spherical coordinates
endData['xval'] = endData['parLSS']*np.cos(endData['parTheta2']*np.pi/180.0)*np.sin(endData['parTheta1']*np.pi/180.0)
endData['yval'] = endData['parLSS']*np.sin(endData['parTheta2']*np.pi/180.0)*np.sin(endData['parTheta1']*np.pi/180.0)
endData['zval'] = endData['parLSS']*np.cos(endData['parTheta1']*np.pi/180.0)
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
endData = WhichEndState(endData)

#Diverge = 0
#V-Shape = 1
#In-line = 2
#Headbutt = 3
#L-shape = 4
#NOTA = 5
    
#Create traces for each end state
traces = [0 for a in range(5)]
opaque = 0.4

fig = make_subplots(rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           ])

fig.add_trace(
    go.Scatter3d(
    x=endData['xval'],
    y=endData['yval'],
    z=endData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=6,
        color=endData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=10)
    )
),
    row=1, col=1
)

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parTheta2'] > 180.0].copy()
endData = endData.sort_values(by=['parLSS','parTheta2','parTheta1'])
endData = endData.reset_index(drop=True)

#Convert to spherical coordinates
endData['xval'] = endData['parLSS']*np.cos(endData['parTheta2']*np.pi/180.0)*np.sin(endData['parTheta1']*np.pi/180.0)
endData['yval'] = endData['parLSS']*np.sin(endData['parTheta2']*np.pi/180.0)*np.sin(endData['parTheta1']*np.pi/180.0)
endData['zval'] = endData['parLSS']*np.cos(endData['parTheta1']*np.pi/180.0)
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
endData = WhichEndState(endData)

fig.add_trace(
    go.Scatter3d(
    x=endData['xval'],
    y=endData['yval'],
    z=endData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=6,
        color=endData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=10)
    )
),
    row=1, col=2
)

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()



'''
#Divergence
divergeData = endData[endData['State'] == 0].copy()
divergeData = divergeData.reset_index(drop=True)
traces[0] = go.Mesh3d(x=divergeData['xval'], 
                     y=divergeData['yval'], 
                     z=divergeData['zval'], 
                     color=divergeData.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )


#V-Shape
VshapeData = endData[endData['State'] == 1].copy()
VshapeData = VshapeData.reset_index(drop=True)
traces[0] = go.Mesh3d(x=VshapeData['xval'], 
                     y=VshapeData['yval'], 
                     z=VshapeData['zval'], 
                     color=VshapeData.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=5.25
                     )

#In-Line
inLineData = endData[endData['State'] == 2].copy()
inLineData = inLineData.reset_index(drop=True)
inLine1Data = inLineData[inLineData['parThetaBW'] <= 180.0].copy()
inLine1Data = inLine1Data.reset_index(drop=True)
inLine2Data = inLineData[inLineData['parThetaBW'] > 180.0].copy()
inLine2Data = inLine2Data.reset_index(drop=True)
traces[1] = go.Mesh3d(x=inLine1Data['xval'], 
                     y=inLine1Data['yval'], 
                     z=inLine1Data['zval'], 
                     color=inLine1Data.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=0.2,
                     alphahull=3
                     )

traces[2] = go.Mesh3d(x=inLine2Data['xval'], 
                     y=inLine2Data['yval'], 
                     z=inLine2Data['zval'], 
                     color=inLine2Data.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=0.2,
                     alphahull=3
                     )

#Headbutt
headbuttData = endData[endData['State'] == 3].copy()
headbuttData = headbuttData.reset_index(drop=True)
traces[3] = go.Mesh3d(x=headbuttData['xval'], 
                     y=headbuttData['yval'], 
                     z=headbuttData['zval'], 
                     color=headbuttData.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=0
                     )


traces[4] = go.Scatter3d(
    x=endData['xval'],
    y=endData['yval'],
    z=endData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=6,
        color=endData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=10)
    )
)

layout = go.Layout(
    #title='Re10: 3D Phase Diagram',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        )
    ),
    showlegend=False,
)

fig = go.Figure(data = traces, layout=layout)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene=dict(
                    xaxis_title=dict(text="x=L sin("+u"\u03B8"+"1) cos("+u"\u03B8"+"2) (R)",font=dict(size=18,family='Times New Roman')),
                    yaxis_title=dict(text="y=L sin("+u"\u03B8"+"1) sin("+u"\u03B8"+"2) (R)",font=dict(size=18,family='Times New Roman')),
                    zaxis_title=dict(text="z=L cos("+u"\u03B8"+"1) (R)",font=dict(size=18,family='Times New Roman')),
                    ),
                    width=1000,height=1000,
                    margin=dict(r=100, b=100, l=100, t=100),
                    font=dict(
                            family="Times New Roman",
                            size=12,
                            color="black"
                            )
            )
#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_zaxes(automargin=True)

camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=2.0)
)

fig.update_layout(scene_camera=camera)
fig.show()

if not os.path.exists(cwd_PYTHON+'/../Figures/3D_Diagram'):
    os.mkdir(cwd_PYTHON+'/../Figures/3D_Diagram')
#fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re2_topview.svg')
print('Done Making Top View')
camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=-2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
#fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re2_botview.svg')
print('Done Making Bot View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=2.0, z=0.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
#fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re2_yposview.svg')
print('Done Making Ypos View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=-2.0, z=0.0),
    #center=dict(x=0, y=0, z=1.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
#fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LTT_Re2_ynegview.svg')
print('Done Making Yneg View')
'''


b4: 1:  177.8
b4: 2:  179.4
b4: BW:  0.0
b4: BW:  337.50000000000006


'\n#Divergence\ndivergeData = endData[endData[\'State\'] == 0].copy()\ndivergeData = divergeData.reset_index(drop=True)\ntraces[0] = go.Mesh3d(x=divergeData[\'xval\'], \n                     y=divergeData[\'yval\'], \n                     z=divergeData[\'zval\'], \n                     color=divergeData.loc[0,\'color\'], \n                     opacity=opaque,\n                     #opacity=1.0,\n                     alphahull=8\n                     )\n\n\n#V-Shape\nVshapeData = endData[endData[\'State\'] == 1].copy()\nVshapeData = VshapeData.reset_index(drop=True)\ntraces[0] = go.Mesh3d(x=VshapeData[\'xval\'], \n                     y=VshapeData[\'yval\'], \n                     z=VshapeData[\'zval\'], \n                     color=VshapeData.loc[0,\'color\'], \n                     opacity=opaque,\n                     #opacity=1.0,\n                     alphahull=5.25\n                     )\n\n#In-Line\ninLineData = endData[endData[\'State\'] == 2].copy()\ninLineData = inLineData.re

#### Combine Configuration and Dynamics into 1 plot
#### Use imshow for configuration (will be a nice background)

In [59]:
%matplotlib inline

def Construct_Image_Data(data):
    #From our color data and initial configurations, construct 2D mesh which has positions and colors for each position
    #If parHx and parHy do not exist, make color white
    xval = np.linspace(-0.5,12.5,14)
    yval = np.linspace(-13.0,9.0,12)
    #print(xval)
    #print(yval)
    
    image = np.ones((12,14,3)) #idx, jdx, r, g, b
    
    for idx in range(len(xval)):
        HxData = data[data['parHx'] == xval[idx]].copy()
        for jdx in range(len(yval)):
            df = HxData[HxData['parHy'] == yval[jdx]].copy()
            #print(len(df['time']))
            if(len(df['time']) == 1):
                image[jdx,idx,:] = [df['color_r'].values,df['color_g'].values,df['color_b'].values]
                
            else:
                image[jdx,idx,:] = [matplotlib.colors.to_rgb('white')[0],matplotlib.colors.to_rgb('white')[1],matplotlib.colors.to_rgb('white')[2]]
    
    return image

def PlotCombined_Config_Dynamics(data,ax):
    #Data Values
    val1    = data['Hx']
    val2    = data['Hy']
    #print(np.amin(data['Hx']))
    #print(np.amax(data['Hx']))
    #sys.exit(0)
    
    image = Construct_Image_Data(data)
    
    #Create
    
    #GENERATE FIGURE
    #Phase Space  2D (Hy vs Hx)
    ax.scatter(val1,val2,s=9,c='k')
    ax.quiver(val1,val2,data['nTx'],data['nTy'],scale=1.5,width=0.005*18,units='xy',color=data['color'])
    ax.scatter(data['parHx'],data['parHy'],c=data['color'],s=9,marker='s')
    #ax.imshow((data['parHy']/2.0,data['parHx']/2.0,data['color_r'],data['color_g'],data['color_b']),origin='lower',cmap=None,extent=(-0.25,6.25,-6.5,4.5),alpha=0.75)
    #ax.pcolor([data['parHx']/2.0,data['parHy']/2.0],color=data['color'],cmap=None)
    #ax.imshow(image,cmap=None,extent=(-0.5,6.5,-7.0,5.0),alpha=0.5,origin='lower')
    
    #Add Swimmer 1 location
    Circle1 = plt.Circle((0.0,0.5),1.0,color='k', clip_on=True)
    ax.add_artist(Circle1)
    Circle2 = plt.Circle((0.0,-2.0),0.5,color='k', clip_on=True)
    ax.add_artist(Circle2)
    
    return ax

allData = pd.read_csv(cwd_PYTHON+'/B_allData_Re2.csv',delimiter=' ')

#Dynamics code
for Theta in ThetaList:
    ThetaData = allData[allData['ThetaA'] == float(Theta)].copy()
    ThetaData = ThetaData.reset_index(drop=True)
    timeList = ThetaData['time'].values.tolist()
    timeSet = set(timeList)
    timeSet = sorted(list(timeSet))
    print(timeSet)
    #2D PHASE SPACE PLOTS (Hy vs Hx)
    #Loop over simulation time
    for idxTime in range(0,201):
        time = round(idxTime*PERIOD,1)
        print('time = ',time)
        data = ThetaData[ThetaData['time'] == time].copy()
        data = data.sort_values(by=['parHx','parHy'])
        data = data.reset_index(drop=True)
        print('Theta = %s: time = %.1f: # of points = %i'%(Theta,time,len(data['parHx'])))
        Theta_BW = data.loc[0,'parTheta']
        #print(data.head(10))
        data['nTx'] = np.cos(data['Theta']*np.pi + np.pi/2.0)
        data['nTy'] = np.sin(data['Theta']*np.pi + np.pi/2.0)
        #Determine which end-state the data is in
        data = WhichEndState(data)
        #2D PHASE SPACE PLOTS (Hy vs Hx) Configurations and Dynamics
        #Figure and subplot for each Theta
        fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(6,6),dpi=250)
        ax.set_title(r'time = %.1f: $\theta$ = %.1f: $H_y$ vs. $H_x$'%(time,Theta_BW),fontsize=15,**csfont)
        ax.set_xlabel(r'$H_x$',fontsize=12,**csfont)
        ax.set_ylabel(r'$H_y$',fontsize=12,**csfont)
        ax.axis([-10.0,10.0,-10.0,10.0])
        ax.set_aspect('equal')
        ax = PlotCombined_Config_Dynamics(data,ax)
        #2D PLOTS
        fig.tight_layout()
        #plt.show()
        #sys.exit(0)
        strDir = cwd_PYTHON+"/../Figures/PhaseSpace/Combined_B/Theta{0}/".format(Theta)
        pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
        fig.savefig(strDir+'Combined_T'+Theta+'_'+str(idxTime)+'.png')
        fig.clf()
    #sys.exit(0)
        
    print('Theta = '+Theta+' is complete!')
    
plt.close()


[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12.0, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13.0, 13.1, 13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15.0, 15.1, 15.2, 15.3, 15.4, 15.5, 15.6, 15.7, 15.8, 15.9, 16.0, 16.1, 16.2, 16.3, 16.4, 16.5, 16.6, 16.7, 16.8, 16.9, 17.0, 17.1, 17.2, 17.3, 17.4, 17.5, 17.6, 17.7, 17.8, 17.9, 18.0, 18.1, 18.2, 1

time =  12.8
Theta = 0.0: time = 12.8: # of points = 104
time =  12.9
Theta = 0.0: time = 12.9: # of points = 104
time =  13.0
Theta = 0.0: time = 13.0: # of points = 104
time =  13.1
Theta = 0.0: time = 13.1: # of points = 104
time =  13.2
Theta = 0.0: time = 13.2: # of points = 104
time =  13.3
Theta = 0.0: time = 13.3: # of points = 104
time =  13.4
Theta = 0.0: time = 13.4: # of points = 104
time =  13.5
Theta = 0.0: time = 13.5: # of points = 104
time =  13.6
Theta = 0.0: time = 13.6: # of points = 104
time =  13.7
Theta = 0.0: time = 13.7: # of points = 104
time =  13.8
Theta = 0.0: time = 13.8: # of points = 104
time =  13.9
Theta = 0.0: time = 13.9: # of points = 104
time =  14.0
Theta = 0.0: time = 14.0: # of points = 104
time =  14.1
Theta = 0.0: time = 14.1: # of points = 104
time =  14.2
Theta = 0.0: time = 14.2: # of points = 104
time =  14.3
Theta = 0.0: time = 14.3: # of points = 104
time =  14.4
Theta = 0.0: time = 14.4: # of points = 104
time =  14.5
Theta = 0.0: time 

time =  5.2
Theta = 22.5: time = 5.2: # of points = 106
time =  5.3
Theta = 22.5: time = 5.3: # of points = 106
time =  5.4
Theta = 22.5: time = 5.4: # of points = 106
time =  5.5
Theta = 22.5: time = 5.5: # of points = 106
time =  5.6
Theta = 22.5: time = 5.6: # of points = 106
time =  5.7
Theta = 22.5: time = 5.7: # of points = 106
time =  5.8
Theta = 22.5: time = 5.8: # of points = 106
time =  5.9
Theta = 22.5: time = 5.9: # of points = 106
time =  6.0
Theta = 22.5: time = 6.0: # of points = 106
time =  6.1
Theta = 22.5: time = 6.1: # of points = 106
time =  6.2
Theta = 22.5: time = 6.2: # of points = 106
time =  6.3
Theta = 22.5: time = 6.3: # of points = 106
time =  6.4
Theta = 22.5: time = 6.4: # of points = 106
time =  6.5
Theta = 22.5: time = 6.5: # of points = 106
time =  6.6
Theta = 22.5: time = 6.6: # of points = 106
time =  6.7
Theta = 22.5: time = 6.7: # of points = 106
time =  6.8
Theta = 22.5: time = 6.8: # of points = 106
time =  6.9
Theta = 22.5: time = 6.9: # of point

time =  19.5
Theta = 22.5: time = 19.5: # of points = 106
time =  19.6
Theta = 22.5: time = 19.6: # of points = 106
time =  19.7
Theta = 22.5: time = 19.7: # of points = 106
time =  19.8
Theta = 22.5: time = 19.8: # of points = 106
time =  19.9
Theta = 22.5: time = 19.9: # of points = 106
time =  20.0
Theta = 22.5: time = 20.0: # of points = 106
Theta = 22.5 is complete!
[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12.0,

time =  12.0
Theta = 45.0: time = 12.0: # of points = 119
time =  12.1
Theta = 45.0: time = 12.1: # of points = 119
time =  12.2
Theta = 45.0: time = 12.2: # of points = 119
time =  12.3
Theta = 45.0: time = 12.3: # of points = 119
time =  12.4
Theta = 45.0: time = 12.4: # of points = 119
time =  12.5
Theta = 45.0: time = 12.5: # of points = 119
time =  12.6
Theta = 45.0: time = 12.6: # of points = 119
time =  12.7
Theta = 45.0: time = 12.7: # of points = 119
time =  12.8
Theta = 45.0: time = 12.8: # of points = 119
time =  12.9
Theta = 45.0: time = 12.9: # of points = 119
time =  13.0
Theta = 45.0: time = 13.0: # of points = 119
time =  13.1
Theta = 45.0: time = 13.1: # of points = 119
time =  13.2
Theta = 45.0: time = 13.2: # of points = 119
time =  13.3
Theta = 45.0: time = 13.3: # of points = 119
time =  13.4
Theta = 45.0: time = 13.4: # of points = 119
time =  13.5
Theta = 45.0: time = 13.5: # of points = 119
time =  13.6
Theta = 45.0: time = 13.6: # of points = 119
time =  13.7
T

...

time =  0.1
Theta = 292.5: time = 0.1: # of points = 117
time =  0.2
Theta = 292.5: time = 0.2: # of points = 117
time =  0.3
Theta = 292.5: time = 0.3: # of points = 117
time =  0.4
Theta = 292.5: time = 0.4: # of points = 117
time =  0.5
Theta = 292.5: time = 0.5: # of points = 117
time =  0.6
Theta = 292.5: time = 0.6: # of points = 117
time =  0.7
Theta = 292.5: time = 0.7: # of points = 117
time =  0.8
Theta = 292.5: time = 0.8: # of points = 117
time =  0.9
Theta = 292.5: time = 0.9: # of points = 117
time =  1.0
Theta = 292.5: time = 1.0: # of points = 117
time =  1.1
Theta = 292.5: time = 1.1: # of points = 117
time =  1.2
Theta = 292.5: time = 1.2: # of points = 117
time =  1.3
Theta = 292.5: time = 1.3: # of points = 117
time =  1.4
Theta = 292.5: time = 1.4: # of points = 117
time =  1.5
Theta = 292.5: time = 1.5: # of points = 117
time =  1.6
Theta = 292.5: time = 1.6: # of points = 117
time =  1.7
Theta = 292.5: time = 1.7: # of points = 117
time =  1.8
Theta = 292.5: time

time =  14.4
Theta = 292.5: time = 14.4: # of points = 117
time =  14.5
Theta = 292.5: time = 14.5: # of points = 117
time =  14.6
Theta = 292.5: time = 14.6: # of points = 117
time =  14.7
Theta = 292.5: time = 14.7: # of points = 117
time =  14.8
Theta = 292.5: time = 14.8: # of points = 117
time =  14.9
Theta = 292.5: time = 14.9: # of points = 117
time =  15.0
Theta = 292.5: time = 15.0: # of points = 117
time =  15.1
Theta = 292.5: time = 15.1: # of points = 117
time =  15.2
Theta = 292.5: time = 15.2: # of points = 117
time =  15.3
Theta = 292.5: time = 15.3: # of points = 117
time =  15.4
Theta = 292.5: time = 15.4: # of points = 117
time =  15.5
Theta = 292.5: time = 15.5: # of points = 117
time =  15.6
Theta = 292.5: time = 15.6: # of points = 117
time =  15.7
Theta = 292.5: time = 15.7: # of points = 117
time =  15.8
Theta = 292.5: time = 15.8: # of points = 117
time =  15.9
Theta = 292.5: time = 15.9: # of points = 117
time =  16.0
Theta = 292.5: time = 16.0: # of points = 1

time =  6.5
Theta = 315.0: time = 6.5: # of points = 112
time =  6.6
Theta = 315.0: time = 6.6: # of points = 112
time =  6.7
Theta = 315.0: time = 6.7: # of points = 112
time =  6.8
Theta = 315.0: time = 6.8: # of points = 112
time =  6.9
Theta = 315.0: time = 6.9: # of points = 112
time =  7.0
Theta = 315.0: time = 7.0: # of points = 112
time =  7.1
Theta = 315.0: time = 7.1: # of points = 112
time =  7.2
Theta = 315.0: time = 7.2: # of points = 112
time =  7.3
Theta = 315.0: time = 7.3: # of points = 112
time =  7.4
Theta = 315.0: time = 7.4: # of points = 112
time =  7.5
Theta = 315.0: time = 7.5: # of points = 112
time =  7.6
Theta = 315.0: time = 7.6: # of points = 112
time =  7.7
Theta = 315.0: time = 7.7: # of points = 112
time =  7.8
Theta = 315.0: time = 7.8: # of points = 112
time =  7.9
Theta = 315.0: time = 7.9: # of points = 112
time =  8.0
Theta = 315.0: time = 8.0: # of points = 112
time =  8.1
Theta = 315.0: time = 8.1: # of points = 112
time =  8.2
Theta = 315.0: time

time =  0.1
Theta = 337.5: time = 0.1: # of points = 119
time =  0.2
Theta = 337.5: time = 0.2: # of points = 119
time =  0.3
Theta = 337.5: time = 0.3: # of points = 119
time =  0.4
Theta = 337.5: time = 0.4: # of points = 119
time =  0.5
Theta = 337.5: time = 0.5: # of points = 119
time =  0.6
Theta = 337.5: time = 0.6: # of points = 119
time =  0.7
Theta = 337.5: time = 0.7: # of points = 119
time =  0.8
Theta = 337.5: time = 0.8: # of points = 119
time =  0.9
Theta = 337.5: time = 0.9: # of points = 119
time =  1.0
Theta = 337.5: time = 1.0: # of points = 119
time =  1.1
Theta = 337.5: time = 1.1: # of points = 119
time =  1.2
Theta = 337.5: time = 1.2: # of points = 119
time =  1.3
Theta = 337.5: time = 1.3: # of points = 119
time =  1.4
Theta = 337.5: time = 1.4: # of points = 119
time =  1.5
Theta = 337.5: time = 1.5: # of points = 119
time =  1.6
Theta = 337.5: time = 1.6: # of points = 119
time =  1.7
Theta = 337.5: time = 1.7: # of points = 119
time =  1.8
Theta = 337.5: time

time =  14.4
Theta = 337.5: time = 14.4: # of points = 119
time =  14.5
Theta = 337.5: time = 14.5: # of points = 119
time =  14.6
Theta = 337.5: time = 14.6: # of points = 119
time =  14.7
Theta = 337.5: time = 14.7: # of points = 119
time =  14.8
Theta = 337.5: time = 14.8: # of points = 119
time =  14.9
Theta = 337.5: time = 14.9: # of points = 119
time =  15.0
Theta = 337.5: time = 15.0: # of points = 119
time =  15.1
Theta = 337.5: time = 15.1: # of points = 119
time =  15.2
Theta = 337.5: time = 15.2: # of points = 119
time =  15.3
Theta = 337.5: time = 15.3: # of points = 119
time =  15.4
Theta = 337.5: time = 15.4: # of points = 119
time =  15.5
Theta = 337.5: time = 15.5: # of points = 119
time =  15.6
Theta = 337.5: time = 15.6: # of points = 119
time =  15.7
Theta = 337.5: time = 15.7: # of points = 119
time =  15.8
Theta = 337.5: time = 15.8: # of points = 119
time =  15.9
Theta = 337.5: time = 15.9: # of points = 119
time =  16.0
Theta = 337.5: time = 16.0: # of points = 1

#### Let's Find the Pair Interaction Force
##### Calculate Velocity, acceleration, then turn into force
##### Idk after yet